In [54]:
import pandas as pd
from google.cloud import bigquery
from flask import Flask
from flask import request, escape
import sys
from ast import literal_eval
import numpy as np
import os

In [ ]:
app = Flask(__name__)


from flask import Flask,render_template,request
 
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/set_base_model')
def form():
    return render_template('form.html')


@app.route('/data', methods = ['POST', 'GET'])
def data():
    if request.method == 'POST':
       

In [2]:
#ONLY FOR LOCAL USE
CREDS = "C:\\Users\\TK\Desktop\\msds434-module5-af0b121e8d76.json"
client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS,project='msds434-module5')


In [22]:
first = 20160801
last = 20160901
dname = 'pickle'
start = str(first)
end = str(last)
name = str(dname)
dset = 'module5'
query = """
    CREATE OR REPLACE MODEL {0}.{1}
    OPTIONS(model_type='logistic_reg',
            input_label_cols=['isBuyer'])
    AS
    SELECT
        IF(totals.transactions IS NULL, 0, 1) as isBuyer,
        IFNULL(totals.pageviews,0) AS pageviews,
        IFNULL(totals.timeOnSite,0) AS timeOnSite,
        IFNULL(totals.newVisits,0) AS isNewVisit,
        IF(device.deviceCategory = 'mobile', 1, 0) as isMobile,
        IF(device.deviceCategory = 'desktop', 1, 0) as isDesktop,
        IF(trafficSource.medium in ('affiliate','cpc','cpm'),1,0) AS isPaidTraffic
    FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE
        _TABLE_SUFFIX BETWEEN @start AND @end
""".format(dset, name)


job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("start", "STRING", start),
        bigquery.ScalarQueryParameter("end", "STRING", end),
    ]
)

query_job = client.query(query, job_config=job_config)  

try: 
  for row in query_job:
    print(row)
except:
  print("ignoring this error: {}".format(sys.exc_info())) #  the model was successfully created

In [4]:
first = 20160801
last = 20160901
dname = 'pickle'
start = str(first)
end = str(last)
name = str(dname)
dset = 'module5'
query = """
        SELECT *
        FROM ML.EVALUATE(MODEL {0}.{1},
        (
        SELECT
        IF(totals.transactions IS NULL, 0, 1) as isBuyer,
        IFNULL(totals.pageviews,0) AS pageviews,
        IFNULL(totals.timeOnSite,0) AS timeOnSite,
        IFNULL(totals.newVisits,0) AS isNewVisit,
        IF(device.deviceCategory = 'mobile', 1, 0) as isMobile,
        IF(device.deviceCategory = 'desktop', 1, 0) as isDesktop,
        IF(trafficSource.medium in ('affiliate','cpc','cpm'),1,0) AS isPaidTraffic
        FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
        WHERE
        _TABLE_SUFFIX BETWEEN @start AND @end
        ),
        STRUCT(0.5 AS threshold)
        )
""".format(dset, name)


job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("start", "STRING", start),
        bigquery.ScalarQueryParameter("end", "STRING", end),
    ]
)

df = client.query(query, job_config=job_config).to_dataframe()

In [5]:
df.head()

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.442623,0.113636,0.984208,0.180844,0.056632,0.964818


In [6]:
first = 20160801
last = 20160901
dname = 'pickle'
start = str(first)
end = str(last)
name = str(dname)
dset = 'module5'
query = """
        SELECT *
        FROM ML.PREDICT(MODEL {0}.{1},
        (
        SELECT
        IF(totals.transactions IS NULL, 0, 1) as isBuyer,
        IFNULL(totals.pageviews,0) AS pageviews,
        IFNULL(totals.timeOnSite,0) AS timeOnSite,
        IFNULL(totals.newVisits,0) AS isNewVisit,
        IF(device.deviceCategory = 'mobile', 1, 0) as isMobile,
        IF(device.deviceCategory = 'desktop', 1, 0) as isDesktop,
        IF(trafficSource.medium in ('affiliate','cpc','cpm'),1,0) AS isPaidTraffic
        FROM
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
        WHERE
        _TABLE_SUFFIX BETWEEN @start AND @end
        ),
        STRUCT(0.5 AS threshold)
        )
""".format(dset, name)


job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("start", "STRING", start),
        bigquery.ScalarQueryParameter("end", "STRING", end),
    ]
)

df = client.query(query, job_config=job_config).to_dataframe()
df2 = pd.DataFrame(df.explode('predicted_isBuyer_probs')['predicted_isBuyer_probs'])
df2 = df2['predicted_isBuyer_probs'].apply(pd.Series)
pivoted = df2.pivot_table(index=df2.index,columns='label',values='prob',aggfunc=np.mean)
pivoted.rename(columns={ pivoted.columns[0]: "Prob_Not_Buyer" ,pivoted.columns[1]: "Prob_IS_Buyer" }, inplace = True)
merged = df.drop('predicted_isBuyer_probs',axis=1)
merged = merged.join(pivoted)
merged


In [7]:
df.head()

,predicted_isBuyer,predicted_isBuyer_probs,isBuyer,pageviews,timeOnSite,isNewVisit,isMobile,isDesktop,isPaidTraffic
0,0,"[{'label': 1, 'prob': 0.04174669190383943}, {'...",0,15,251,0,0,1,1
1,0,"[{'label': 1, 'prob': 0.019837634351636692}, {...",0,14,173,1,0,1,0
2,0,"[{'label': 1, 'prob': 0.07780939479203088}, {'...",0,19,1124,0,0,1,0
3,0,"[{'label': 1, 'prob': 0.042352119906786265}, {...",0,14,127,0,0,1,0
4,0,"[{'label': 1, 'prob': 0.06768425570774214}, {'...",0,15,1281,0,0,1,0


In [30]:
df2 = pd.DataFrame(df.explode('predicted_isBuyer_probs')['predicted_isBuyer_probs'])
df2 = df2['predicted_isBuyer_probs'].apply(pd.Series)
pivoted = df2.pivot_table(index=df2.index,columns='label',values='prob',aggfunc=np.mean)
pivoted.rename(columns={ pivoted.columns[0]: "Prob_Not_Buyer" ,pivoted.columns[1]: "Prob_IS_Buyer" }, inplace = True)
merged = df.drop('predicted_isBuyer_probs',axis=1)
merged = merged.join(pivoted)
merged

,predicted_isBuyer_probs
0,"{'label': 1, 'prob': 0.04174669190383943}"
0,"{'label': 0, 'prob': 0.9582533080961606}"
1,"{'label': 1, 'prob': 0.019837634351636692}"
1,"{'label': 0, 'prob': 0.9801623656483633}"
2,"{'label': 1, 'prob': 0.07780939479203088}"
...,...
77443,"{'label': 0, 'prob': 0.5994330460128433}"
77444,"{'label': 1, 'prob': 0.8317764984520473}"
77444,"{'label': 0, 'prob': 0.16822350154795274}"
77445,"{'label': 1, 'prob': 0.9989234415358613}"


In [ ]:
if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))